In [ ]:
import numpy as np
from collections import defaultdict
import random

# Regole del Blackjack

![](img/blackjack.png)


* Si usano le carte **francesi**.
* Le figure (**J**, **K**, **Q**) valgono 10, l'asso può valere 1 o 11. Le altre carte hanno il valore naturale.
* All'inizio vengono assegnate due carte al banco e due al giocatore. La prima carta del banco è scoperta.
* Il punteggio è dato dalla somma delle carte (con l'asso che può valere 1 o 11). 
* Un punteggio superiore a 21 non è valido. In questo caso si dice che il giocatore (o il banco) ha *sballato*.
* Se le due carte iniziali sono un asso e una carta con valore 10 (10 o figura) si ha il **Blackjack**.
* Il giocatore può chiedere, una per volta, tutte le carte che vuole.
* Se il giocatore non ha sballato (vittoria del banco) il banco estrae le carte usando una strategia fissata, ad esempio fermandosi quando il punteggio è almeno 17.
* Alla fine vince chi ha un punteggio maggiore, con possibile parità.

In [ ]:
class BlackjackEnv:
    cards = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10]

    def __init__(self):
        self.done = True

    def _take_card(self):
        ''' Estrae una carta'''
        return random.choice(self.cards)

    def _score(self, cards):
        if sum(cards) > 21:
            # sballato
            return 0
        elif len(cards) == 2 and 1 in cards and 10 in cards:
            # Blackjack
            return 22
        else:
            return sum(cards) if not self._is_usable_ace(cards) else sum(cards) + 10

    def _is_usable_ace(self, cards):
        ''' Se c'è un asso utilizzabile come 11'''
        return 1 in cards and sum(cards) <= 11

    def _dealer_get_cards(self):
        ''' Implementa la policy del banco'''
        while 0 < self._score(self.dealer_cards) <= 16:
            # prendi ancora una carta se non hai sballato
            # e il punteggio è minore o uguale a 16
            self.dealer_cards.append(self._take_card())

        return self.dealer_cards

    def reset(self):
        ''' Stato iniziale della partita'''
        self.done = False
        # due carte ciascuno
        self.dealer_cards = [self._take_card(), self._take_card()]
        self.player_cards = [self._take_card(), self._take_card()]

        state = (self.dealer_cards[0], self._score(self.player_cards), self._is_usable_ace(self.player_cards))

        return state

    def step(self, ask_card):
        ''' Azione di gioco'''
        if self.done:
            raise Exception('Il gioco è terminato. Chiamare reset per cominciare un nuovo gioco')

        # Il giocatore chiede una carta o si ferma.
        # Sono le due azioni possibili
        if ask_card:
            # giocatore richiede nuova carta
            self.player_cards.append(self._take_card())
            if self._score(self.player_cards) > 0:
                # punteggio valido dopo la nuova estrazione
                reward = 0
                self.done = False
            else:
                # il giocatore ha sballato, vince il banco
                reward = -1
                self.done = True
        else:
            # giocatore si ferma, tocca al banco
            self.done = True
            player_score = self._score(self.player_cards)
            dealer_score = self._score(self._dealer_get_cards())
            # punteggio del giocatore
            if player_score < dealer_score:
                reward = -1
            elif player_score > dealer_score:
                reward = 1
            else:
                reward = 0
                
        if self.done:
            # stato terminale
            state = None, None, None
        else:
            state = (self.dealer_cards[0], self._score(self.player_cards), self._is_usable_ace(self.player_cards))

        return state, reward, self.done

In [ ]:
def generate_episode(environment, policy):
    ''' Genera un episodio ustilizzando la policy passata come argomento'''
    states, actions, rewards = [], [], []
    state = environment.reset()
    done = False
    while not done:
        action = policy.get_action(state)
        new_state, reward, done = environment.step(action)
        states.append(state)
        actions.append(action)
        rewards.append(reward)
        state = new_state

    return states, actions, rewards

In [ ]:
class PlayerPolicy:
    ''' Strategia di gioco del giocatore'''
    def get_action(self, state):
        dealer_first_card, player_score, is_usable_ace = state
        if player_score <= 17:
            # chiede un'altra carta
            return True
        else:
            # sta
            return False

In [ ]:
def mc_v_estimation(environment, policy, num_episodes, first_visit, gamma):
    '''Stima la funzione di valore di stato V di una policy con metodo Montecarlo

    args
    ----
    environment  - un ambiente Gym
    policy       - un oggetto che implementa il metodo get_action.
                   get_action prende come argomento uno stato e ritorna un'azione
    num_episodes - numero di episodi da generare
    first_visit  - se True implementa first_visit Monecarlo, altrimenti every visit
    gamma        - fattore di sconto
    
    rets
    ----
    V            - un dizionario che rappresenta la funzione di valore di stato V,
                   nella forma {state: value, ...}'''
    return V

In [ ]:
V_mc = mc_v_estimation(BlackjackEnv(), PlayerPolicy(), num_episodes=100_000, first_visit=True, gamma=0.99)

In [ ]:
V_mc

In [ ]:
def td_0_v_estimation(environment, policy, num_episodes, gamma, alpha):
    '''Stima la funzione di valore di stato V di una policy con metodo TD(0)

    args
    ----
    environment  - un ambiente Gym
    policy       - un oggetto che implementa il metodo get_action.
                   get_action prende come argomento uno stato e ritorna un'azione
    num_episodes - numero di episodi da generare
    gamma        - fattore di sconto
    alpha        - coefficiente di aggiornamento
    
    rets
    ----
    V            - un dizionario che rappresenta la funzione di valore di stato V,
                   nella forma {state: value, ...}'''
                   
    return V

In [ ]:
V_td = td_0_v_estimation(BlackjackEnv(), PlayerPolicy(), num_episodes=100_000, gamma=0.9, alpha=0.1)

In [ ]:
V_td

In [ ]:
def td_n_v_estimation(n, environment, policy, num_episodes, gamma, alpha):
    '''Stima la funzione di valore di stato V di una policy con metodo n-step TD

    args
    ----
    n            - il valore di n per il metodo TD
    environment  - un ambiente Gym
    policy       - un oggetto che implementa il metodo get_action.
                   get_action prende come argomento uno stato e ritorna un'azione
    num_episodes - numero di episodi da generare
    first_visit  - se True implementa first_visit Monecarlo, altrimenti every visit
    gamma        - fattore di sconto
    alpha        - coefficiente di aggiornamento
    
    rets
    ----
    V            - un dizionario che rappresenta la funzione di valore di stato V,
                   nella forma {state: value, ...}'''


    return V 

In [ ]:
V_td_n = td_n_v_estimation(5, BlackjackEnv(), PlayerPolicy(), num_episodes=100_000, gamma=0.9, alpha=0.1)

In [ ]:
V_td_n